In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 945.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.4 MB/s et

# 基于gradio实现的UI包括5个部分：

- RAG文本对话tab，这里主要是方便后续扩展到多模型的问答，将纯文本的问答作为一个tab，基于图像的多模态问答可以新建一个tab集成到该项目下。
- 模型选择，本项目支持调用第三方平台的模型，比如百度的文心一言，也支持自己部署开源或微调大模型的调用，所以支持了模型的选择。
- 聊天历史展示，展示过往了历史聊天
- 问答输入，通过该文本框输入问题，后台会根据问题先从本地知识库中检索，然后调用大模型综合回答问题。
- 本地文件上传，本地知识库可以通过项目启动初始化，也支持用户聊天过程中上传。

In [ ]:
import os
import time
import gradio as gr

def add_text(history, text):
    print("add_text")
    history = history + [(text, None)]
    return history, gr.update(value='', interactive=True)

def upload_file(history, file):
    print("upload file")
    """
        todo:
        1. 上传文档
        2. 文档切割
        3. 文档向量化
        4. 检索
    """
    history = history + [((file.name, ), None)]
    print("upload_file,", history)
    return history


def chat(history):
    print("chat")
    print("upload_file,", history)

    msg = history[-1][0]
    """
    # todo:
        1. 根据用户的提问，调用检索服务，获取与问题相关本地知识库信息
        2. 调用大模型，对信息进行整合
        3. 解析大模型结果，并返回
    """
    ans = ''

    history[-1][1] = ''
    for ch in ans:
        history[-1][1] += ch
        time.sleep(0.01)
        yield history[-1:]


with gr.Blocks() as demo:
    with gr.Tab("RAG文本对话"):
        model_list = ["文心一言大模型", "chatglm3本地部署大模型"]
        input_model_list = gr.Dropdown(choices=model_list, label='模型选择')

        chatbot = gr.Chatbot(
            [],
            elem_id='chatbot',
            bubble_full_width=False
        )

        with gr.Row():
            txt = gr.Textbox(
                scale=4,
                show_label=False,
                placeholder='输入文本并回车，后者上传本地文件',
                container=False
            )
            btn = gr.UploadButton("文件", file_types=[])

        txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
            chat, chatbot, chatbot
        )
        file_msg = btn.upload(upload_file, [chatbot, btn], [chatbot], queue=False).then(
            chat, chatbot, chatbot
        )

demo.queue()

if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a90e323036a407c05a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
